<a href="https://colab.research.google.com/github/dw3101/ELIZA/blob/main/ELIZA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install textblob

In [ ]:
import re
import random
from textblob import TextBlob

In [ ]:
#pronouns swapping helper method:
def swap_pronouns(text):
  swaps = {"i":"you", "me":"you", "my":"your", "am":"are", "you":"I", "your":"my", "mine":"yours", "myself":"yourself"
  }
words = text.lower().split()
return ''. join([swaps[word] if word in swaps else word for word in words])


NameError: name 'text' is not defined